读写文件是最常见的IO操作。Python内置了读写文件的函数，用法和C是兼容的。
在磁盘上读写文件的功能都是由操作系统提供的，现代操作系统不允许普通的程序直接操作磁盘，所以，读写文件就是请求操作系统打开一个文件对象，然后，通过操作系统提供的接口从这个文件对象中读取数据（读文件），或者把数据写入这个文件对象（写文件）。

读文件

要以读文件的模式打开一个文件对象，使用Python内置的open()函数，传入文件名和标示符：

In [2]:
f = open('test.txt', 'r') #标示符'r'表示读，这样，我们就成功地打开了一个文件

如果文件不存在，open()函数就会抛出一个IOError的错误，并且给出错误码和详细的信息告诉你文件不存在
如果文件打开成功，接下来，调用read()方法可以一次读取文件的全部内容，Python把内容读到内存，用一个str对象表示：

In [3]:
f.read()

'123\nabc'

最后一步是调用close()方法关闭文件。文件使用完毕后必须关闭，因为文件对象会占用操作系统的资源，并且操作系统同一时间能打开的文件数量也是有限的：

In [4]:
f.close()

由于文件读写时都有可能产生IOError，一旦出错，后面的f.close()就不会调用。所以，为了保证无论是否出错都能正确地关闭文件，我们可以使用try ... finally来实现：

In [5]:
try:
    f = open('error.txt', 'r')
    print f.read()
finally:
    if f:
        f.close()

IOError: [Errno 2] No such file or directory: '/path/to/file'

但是每次都这么写实在太繁琐，所以，Python引入了with语句来自动帮我们调用close()方法：

In [6]:
with open('error', 'r') as f:
    print f.read()

IOError: [Errno 2] No such file or directory: '/path/to/file'

调用read()会一次性读取文件的全部内容，如果文件有10G，内存就爆了，所以，要保险起见，可以反复调用read(size)方法，每次最多读取size个字节的内容。另外，调用readline()可以每次读取一行内容，调用readlines()一次读取所有内容并按行返回list。因此，要根据需要决定怎么调用。

如果文件很小，read()一次性读取最方便；如果不能确定文件大小，反复调用read(size)比较保险；如果是配置文件，调用readlines()最方便：

In [7]:
f = open('test.txt', 'r')
for line in f.readlines():
    print(line.strip()) # 把末尾的'\n'删掉

123
abc


二进制文件

前面读取的是文本文件，并且是ASCII编码的文本文件。要读取二进制文件，比如图片、视频等等，用'rb'模式打开文件即可：

In [9]:
f = open('test.jpg', 'rb')
f.read()

'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x00XExif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x02\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00&\x00\x00\x00\x00\x00\x03\xa0\x01\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\xa0\x02\x00\x04\x00\x00\x00\x01\x00\x00\x01\xdb\xa0\x03\x00\x04\x00\x00\x00\x01\x00\x00\x01\x0c\x00\x00\x00\x00\xff\xed\x008Photoshop 3.0\x008BIM\x04\x04\x00\x00\x00\x00\x00\x008BIM\x04%\x00\x00\x00\x00\x00\x10\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~\xff\xc0\x00\x11\x08\x01\x0c\x01\xdb\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvw

字符编码

要读取非ASCII编码的文本文件，就必须以二进制模式打开，再解码。比如GBK编码的文件：

In [15]:
f = open('gbk.txt', 'rb')
u = f.read().decode('gbk')
u
#print u

u'\u6d4b\u8bd5'

写文件

写文件和读文件是一样的，唯一区别是调用open()函数时，传入标识符'w'或者'wb'表示写文本文件或写二进制文件：

In [16]:
f = open('test2.txt', 'w')
f.write('Hello, world!')
f.close()

In [17]:
f = open('test2.txt', 'r')
f.read()

'Hello, world!'

你可以反复调用write()来写入文件，但是务必要调用f.close()来关闭文件。当我们写文件时，操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲的时候再慢慢写入。只有调用close()方法时，操作系统才保证把没有写入的数据全部写入磁盘。忘记调用close()的后果是数据可能只写了一部分到磁盘，剩下的丢失了。所以，还是用with语句保险：

In [18]:
with open('test3.txt', 'w') as f:
    f.write('Hello, Python!')

文件目录操作

In [20]:
import os
os.getcwd() #该函数的返回值是当前的绝对路径

'/Users/mymac'

In [22]:
#listdir(path_str)函数的返回值是一个列表，列表中包含路径path_str（字符串变量）下所有文件和目录的名称。
os.listdir(path_str)

['.anaconda',
 '.android',
 '.bash_history',
 '.bash_profile',
 '.bash_profile-anaconda.bak',
 '.bash_sessions',
 '.CFUserTextEncoding',
 '.conda',
 '.condarc',
 '.config',
 '.cups',
 '.DS_Store',
 '.gitconfig',
 '.idlerc',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.matplotlib',
 '.oracle_jre_usage',
 '.pylint.d',
 '.sogouinput',
 '.spyder',
 '.spyder2',
 '.ssh',
 '.subversion',
 '.Trash',
 '.vscode',
 '.wns',
 '.yunpan',
 '360\xe4\xba\x91\xe7\x9b\x98',
 'anaconda',
 'AnacondaProjects',
 'Applications',
 'Applications (Parallels)',
 'Desktop',
 'Documents',
 'Downloads',
 'gbk.txt',
 'Library',
 'Movies',
 'Music',
 'Pictures',
 'Public',
 'PycharmProjects',
 'Python_3.ipynb',
 'Python_5.ipynb',
 'Python_Share.slides.html',
 'Python_Share1.slides.html',
 'Python_Share2.slides.html',
 'Python_Share3.slides.html',
 'test.ipynb',
 'test.jpg',
 'test.slides.html',
 'test.txt',
 'test2.txt',
 'test3.txt',
 'Untitled.ipynb',
 'untitled0.py',
 'Untitled1.ipynb',
 'untitled1.txt',
 'U

In [23]:
#remove()函数用来删除"file_name"字符串所指定的文件，但是如果file_name所指定文件名称不存在就会引起错误。
os.remove("file_name")

OSError: [Errno 2] No such file or directory: 'file_name'

In [24]:
try:
    os.remove("file_name")
except OSError,e:
    print('error...but continue')
print('whatever...')

error...but continue
whatever...


Python和很多高级语言一样，内置了一套try...except...finally...的错误处理机制

当我们认为某些代码可能会出错时，就可以用try来运行这段代码，如果执行出错，则后续代码不会继续执行，而是直接跳转至错误处理代码，即except语句块，执行完except后，如果有finally语句块，则执行finally语句块，至此，执行完毕。
下面的代码在计算10 / 0时会产生一个除法运算错误：

In [25]:
try:
    print 'try...'
    r = 10 / 0
    print 'result:', r
except ZeroDivisionError, e:
    print 'except:', e
finally:
    print 'finally...'
print 'END'

try...
except: integer division or modulo by zero
finally...
END


从输出可以看到，当错误发生时，后续语句print 'result:', r不会被执行，except由于捕获到ZeroDivisionError，因此被执行。最后，finally语句被执行。然后，程序继续按照流程往下走。

如果把除数0改成2，则：

In [26]:
try:
    print 'try...'
    r = 10 / 2
    print 'result:', r
except ZeroDivisionError, e:
    print 'except:', e
finally:
    print 'finally...'
print 'END'

try...
result: 5
finally...
END


由于没有错误发生，所以except语句块不会被执行，但是finally如果有，则一定会被执行（可以没有finally语句）。